In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import warnings
from math import floor

import SALib
import numpy as np
import pandas as pd
from tqdm import tqdm
from SALib.sample import saltelli
from SALib.analyze import sobol
from numba import jit, njit, prange, vectorize, float64
from multiprocessing import Pool

In [3]:
warnings.filterwarnings("ignore")

### 1. Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги, 25 баллов

In [53]:
def test_func(a, b, c):
    res = (a/2)**2 + (b/3)**2 - c
    
    return res

In [59]:
problem = {
    'num_vars': 3,
    'names': ['a', 'b', 'c'],
    'bounds': [[0, 1], [0, 1], [0, 1]]
}
n = 2 ** 20
param_values = saltelli.sample(problem, n)

/var/folders/3x/vv95p66j6658cvlngdhhfd1r0000gq/T/ipykernel_94458/809134031.py:7: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)


In [60]:
%%time
y_vals = []

for a, b, n in tqdm(param_values):
    y_vals.append(test_func(a, b, n))

y_vals = np.array(y_vals)

100%|██████████| 8388608/8388608 [00:22<00:00, 374340.73it/s]


CPU times: user 21.7 s, sys: 672 ms, total: 22.3 s
Wall time: 23 s


In [61]:
si = sobol.analyze(problem, y_vals)
print(si['S1'])
print(si['ST'])
print("a-b", si['S2'][0, 1])
print("a-c", si['S2'][0, 2])
print("b-c", si['S2'][1, 2])

[0.06173781 0.01219512 0.92606707]
[0.0617378  0.01219512 0.92606707]
a-b -8.846784051858014e-10
a-c -1.646618397188604e-10
b-c -7.282694447496851e-10


При большом количестве сэмплов (равном степени 2), получилось добиться того, что индекс стали близки нулю (хоть они формально все еще в отрицательной части)

### 2. Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba)

In [96]:
def dummy_prime(n):
    is_prime = True
    for i in range(2, n):
        if n % i == 0:
            is_prime = False
    return is_prime


@njit(parallel=True)
def numba_dummy_prime(n):
    is_prime = True
    for i in range(2, n):
        if n % i == 0:
            is_prime = False
    return is_prime

In [99]:
start = time.time()
dummy_prime(1000000000)
print("Время выполнения:", time.time() - start)

Время выполнения: 63.28783822059631


In [98]:
start = time.time()
numba_dummy_prime(1000000000)
print("Время выполнения ускоренного кода:", time.time() - start)

Время выполнения ускоренного кода: 2.932878255844116


Видно существенное ускорение. Такой прирост связан с тем, что используется крайне простая функция. Во время тестов с более сложной функцией подсчета геохэша (пример ниже) такое ускорение не наблюдалось

### 3. Попробуйте добавить параллелизм в вычисления, 25 баллов

В этом разделе будем параллельно вычислять геохэши, так как вычисление геохэша по заданным координатам - довольно важная задача при работе с гео-данными.
Для вычислений разобьем массив координат на батчи и запустим несколько процессов параллельно

In [6]:
from geohash import chunk_encode, encode

In [7]:
TEST_COORDS = (55.746132, 37.694568)
print(encode(*TEST_COORDS))

210303213022011230


In [8]:
problem = {
    'num_vars': 2,
    'names': ['lat', 'lon'],
    'bounds': [[55.0, 56.0], [37.0, 38.0]]
}
n = 2 ** 20
param_values = saltelli.sample(problem, n)
print("Количество координат для перебора:", len(param_values))

Количество координат для перебора: 6291456


In [12]:
start = time.time()
chunk_encode((-1, param_values))
print("Время последовательного перебора:", time.time() - start)

Время последовательного перебора: 140.99510884284973


In [13]:
PROC_CNT = 4

dataset_parts = np.split(param_values, PROC_CNT)
start = time.time()

with Pool(processes=PROC_CNT) as pool:
    pool.map(chunk_encode, [(i, itm) for i, itm in enumerate(dataset_parts)])

print(f"Время параллельного перебора при разделении на {PROC_CNT} параллельных процесса:", time.time() - start)

Время параллельного перебора при разделении на 4 параллельных процесса: 52.75429821014404


Видно, что после разделения обработки данных на параллельные процессы, время работы сильно сократилось (даже с учетом накладных расходов на выделение ресурсов при создании процессов)